# D. State stabilization in a noisy environment with Jaynes-Cummings controls

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
from feedback_grape.fgrape import optimize_pulse
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [3]:
N_cav = 30

## Here, dividing alpha into real and imaginary parts complicates the optimization and converges at 0.89 while if we do not use the imaginary part it converges at 0.999

In [4]:
def qubit_unitary(alphas):
    alpha = alphas[0] + 0j # Only real part
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [5]:
def qubit_cavity_unitary(betas):
    beta = betas[0] + 0j # Only real part
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

### povm_measure_operator (callable): <br>
    - It should take a measurement outcome and list of params as input
    - The measurement outcome options are either 1 or -1

In [6]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2 * identity(2*N_cav)
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [7]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [8]:
rho_target.shape

(60, 60)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized

In [9]:
# Answer: should one normalize within the optimization just in case?
# --> no so that user is not misled into thinking that his unnormalized state
# is working properly
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target,
    +1,
    povm_measure_operator(+1, params=[0.1, -3 * jnp.pi / 2]),
    povm_measure_operator(-1, params=[0.1, -3 * jnp.pi / 2]),
    evo_type="density"
)

Array(0.09204736, dtype=float64)

In [10]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, evo_type="density"))

1.0000002330219993


### Without dissipation

In [11]:
# Here the loss directly corressponds to the -fidelity (when converging) because log(1) is 0 and
# the algorithm is choosing params that makes the POVM generate prob = 1
from feedback_grape.fgrape import Gate, Decay

measure = Gate(
    gate=povm_measure_operator,
    initial_params=[0.058, jnp.pi / 2],  # gamma and
    measurement_flag=True,
)

qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=[jnp.pi / 3],  # alpha (only real part)
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=[jnp.pi / 3],  # beta (only real part)
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)


for reward_weights in [[1.0,1.0],[0.0,1.0]]:
    system_params = [measure, qub_unitary, qub_cav]
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-16,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_batch_size=16,
        eval_time_steps=5,
    )

    print(f"reward weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n fidelity_each_timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward weights: [1.0, 1.0]
 fidelity@t=2: [0.79705274 0.79705274 0.79705274 0.79705274 0.79705274 0.79705274
 0.79705274 0.79705274 0.79705274 0.79705274 0.79705274 0.79705274
 0.79705274 0.79705274 0.79705274 0.79705274]
 fidelity_each_timestep: [1.00000023 0.99999788 0.79705274 0.40586777 0.17199196 0.07169528]

reward weights: [0.0, 1.0]
 fidelity@t=2: [0.80358979 0.80358979 0.80358979 0.80358979 0.80358979 0.80358979
 0.80358979 0.80358979 0.80358979 0.80358979 0.80358979 0.80358979
 0.80358979 0.80358979 0.80358979 0.80358979]
 fidelity_each_timestep: [1.00000023 0.00717262 0.80358979 0.00100963 0.41391245 0.00473293]



In [12]:
result.fidelity_each_timestep

[Array([1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023], dtype=float64),
 Array([0.00717262, 0.00717262, 0.00717262, 0.00717262, 0.00717262,
        0.00717262, 0.00717262, 0.00717262, 0.00717262, 0.00717262,
        0.00717262, 0.00717262, 0.00717262, 0.00717262, 0.00717262,
        0.00717262], dtype=float64),
 Array([0.80358979, 0.80358979, 0.80358979, 0.80358979, 0.80358979,
        0.80358979, 0.80358979, 0.80358979, 0.80358979, 0.80358979,
        0.80358979, 0.80358979, 0.80358979, 0.80358979, 0.80358979,
        0.80358979], dtype=float64),
 Array([0.00041852, 0.00041852, 0.00041852, 0.00041852, 0.00041852,
        0.00041852, 0.00041852, 0.00041852, 0.00041852, 0.00041852,
        0.00041852, 0.00041852, 0.00041852, 0.00041852, 0.00041852,
        0.00987638], dtype=float64),
 Array([0.43614574, 0.43614574, 0.43

In [13]:
result.optimized_trainable_parameters["initial_params"]

[Array([0.03819765, 2.39149772], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [14]:
result.returned_params

[[Array([[0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772],
         [0.03819765, 2.39149772]], dtype=float64),
  Array([[-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
         [-2.78672891e-06],
   

In [15]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000002330219993
fidelity of state 0: 0.004827032210083218
fidelity of state 1: 0.004827032210083218
fidelity of state 2: 0.004827032210083218
fidelity of state 3: 0.004827032210083218
fidelity of state 4: 0.004827032210083218
fidelity of state 5: 0.004827032210083218
fidelity of state 6: 0.004827032210083218
fidelity of state 7: 0.004827032210083218
fidelity of state 8: 0.004827032210083218
fidelity of state 9: 0.002883930772113769
fidelity of state 10: 0.004827032210083218
fidelity of state 11: 0.004827032210083218
fidelity of state 12: 0.004827032210083218
fidelity of state 13: 0.004827032210083218
fidelity of state 14: 0.002883930772113769
fidelity of state 15: 0.007207546862241162


### With Dissipation

In [16]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
# Question: Here I should not use any sort of Hamiltonian or tspan or something?
decay = Decay(
    c_ops=[tensor(identity(N_cav), jnp.sqrt(0.10) * sigmam())],
)
system_params = [decay, measure, qub_unitary, qub_cav]

for reward_weights in [[1.0,1.0],[0.0,1.0]]:
    result = optimize_pulse(
        U_0=rho_target,
        C_target=rho_target,
        system_params=system_params,
        num_time_steps=2,
        reward_weights=reward_weights,
        mode="lookup",
        goal="fidelity",
        max_iter=1000,
        convergence_threshold=1e-6,
        learning_rate=0.005,
        evo_type="density",
        batch_size=1,
        eval_batch_size=16,
        eval_time_steps=5,
    )

    print(f"reward weights: {reward_weights}\n fidelity@t=2: {result.fidelity_each_timestep[2]}\n fidelity_each_timestep: {jnp.mean(jnp.array(result.fidelity_each_timestep), axis=1)}\n")

reward weights: [1.0, 1.0]
 fidelity@t=2: [0.65138342 0.65138342 0.65138342 0.65138342 0.65138342 0.65138342
 0.65138342 0.65138342 0.65138342 0.65138342 0.65138342 0.65138342
 0.65138342 0.65138342 0.65138342 0.65138342]
 fidelity_each_timestep: [1.00000023 0.90483583 0.65138342 0.29924073 0.09127508 0.03480664]

reward weights: [0.0, 1.0]
 fidelity@t=2: [0.73379623 0.73379623 0.17888064 0.73379623 0.73379623 0.73379623
 0.73379623 0.73379623 0.73379623 0.73379623 0.73379623 0.73379623
 0.73379623 0.73379623 0.73379623 0.73379623]
 fidelity_each_timestep: [1.00000023e+00 7.97549985e-04 6.99114003e-01 9.50845559e-03
 1.46522815e-01 1.06326590e-01]



In [17]:
result.fidelity_each_timestep

[Array([1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023, 1.00000023, 1.00000023, 1.00000023, 1.00000023,
        1.00000023], dtype=float64),
 Array([0.00079755, 0.00079755, 0.00079755, 0.00079755, 0.00079755,
        0.00079755, 0.00079755, 0.00079755, 0.00079755, 0.00079755,
        0.00079755, 0.00079755, 0.00079755, 0.00079755, 0.00079755,
        0.00079755], dtype=float64),
 Array([0.73379623, 0.73379623, 0.17888064, 0.73379623, 0.73379623,
        0.73379623, 0.73379623, 0.73379623, 0.73379623, 0.73379623,
        0.73379623, 0.73379623, 0.73379623, 0.73379623, 0.73379623,
        0.73379623], dtype=float64),
 Array([0.00783445, 0.00783445, 0.03461856, 0.00783445, 0.00783445,
        0.00783445, 0.00783445, 0.00783445, 0.00783445, 0.00783445,
        0.00783445, 0.00783445, 0.00783445, 0.00783445, 0.00783445,
        0.00783445], dtype=float64),
 Array([0.15282499, 0.15282499, 0.05

In [18]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.0000002330219993
fidelity of state 0: 0.001521773449554806
fidelity of state 1: 0.13783559689229252
fidelity of state 2: 0.019578658338493427
fidelity of state 3: 0.13783559689229252
fidelity of state 4: 0.13783559689229252
fidelity of state 5: 0.13783559689229252
fidelity of state 6: 0.13783559689229252
fidelity of state 7: 0.13783559689229252
fidelity of state 8: 0.013048921660060745
fidelity of state 9: 0.13783559689229252
fidelity of state 10: 0.13783559689229252
fidelity of state 11: 0.013048921660060745
fidelity of state 12: 0.13783559689229252
fidelity of state 13: 0.13783559689229252
fidelity of state 14: 0.13783559689229252
fidelity of state 15: 0.13783559689229252


In [19]:
result.returned_params

[[Array([[0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829],
         [0.0376334 , 2.39761829]], dtype=float64),
  Array([[-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
         [-5.88313157e-05],
   

In [20]:
result.optimized_trainable_parameters["initial_params"]

[Array([0.0376334 , 2.39761829], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]